In [2]:
import numpy as np
import pandas as pd
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
   

In [14]:
def ChinaDaily_WebData(part):

    link_list = []
    name_list = []
    time_list = []
    my_url = "http://www.chinadaily.com.cn/"+ part
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()

    # doing the html parsing
    page_soap = soup(page_html, "html.parser")
    
    # get the number of the pages
    try:
        pages = page_soap.findAll("div",{"selectpage res-m"})
        page = pages[0].span.a.text
        page_num = page[21:]
        
    except:
        pages = page_soap.findAll("div",{"selectpage"})
        page = pages[0].span.a.text
        page_num = page[21:]
        
        
       

    #I have to count mannully for the number of the page
    for i in range(1,int(page_num)+1):
        my_url = "http://www.chinadaily.com.cn/"+ part + "page_" +str(i) + ".html"
        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close()
        # doing the html parsing
        page_soap = soup(page_html, "html.parser")
        #grab each product
        containers = page_soap.findAll("div",{"mb10 tw3_01_2 "})

        for container in containers:
            link_to_article = container.h4.a["href"][2:]
            link_list.append(link_to_article)
            name_of_article = container.h4.a.text
            name_list.append(name_of_article)
            time_of_article = container.b.text
            time_list.append(time_of_article)

    author_list = [] 
    content_list = []
    for link in link_list:

        my_url = "http://" + link
        uClient = uReq(my_url)
        page_html = uClient.read()
        uClient.close()
        # doing the html parsing
        page_soap = soup(page_html, "html.parser")
        #get the authors name
        info = page_soap.find("div",{"info"})
        try:
            info_list = info.text.split("|")
            author = info_list[0]
            author_list.append(author)
        except:
            author_list.append(np.nan)


        try:
            #if there is a parsing for filping pages
            num_content = page_soap.find("span",{"pageno"})
            #get the number of pages
            num=num_content.a.text.split("\n")[1].split("/")[-1]


            #get the content of the webpage
            words = ""
            # going through each pages of the article
            for i in range(1,int(num)+1):

                my_url = "http://" + link[:-5] +"_"+str(i)+".html"


                uClient = uReq(my_url)
                page_html = uClient.read()
                uClient.close()
                # doing the html parsing
                page_soap = soup(page_html, "html.parser")

                content = page_soap.findAll("p",)
                content = content[:-1]
                for item in content:
                    words += item.text



        except:
            words = ""
            content = page_soap.findAll("p",)
            content = content[:-1]
            for item in content:
                words += item.text



        #pend the if the content has a next page
        #continue_run = page_soap.find("span",{"next"})


        content_list.append(words)

    df = pd.DataFrame({'title of the article': name_list,
                            'link of the article': link_list,
                            'author': author_list,
                            'content': content_list,                           
                            'time of the article': time_list})
    return df
    
   

## this function can scrap any part of the data from china daily
## just select the articles from which section.
## the example down below
## the result will return a pandas dataframe with the requested section of the article

In [ ]:
ChinaDaily_WebData("/sports/golf/")

13
